In [3]:
import numpy as np
import time

# Objective function: Rosenbrock function
def rosenbrock(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# Exact gradient of the Rosenbrock function
def rosenbrock_gradient(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

# Exact Hessian of the Rosenbrock function
def rosenbrock_hessian(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

# Approximate gradient using finite differences
def approximate_gradient(x, eps=1e-5):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += eps
        x2[i] -= eps
        grad[i] = (rosenbrock(x1) - rosenbrock(x2)) / (2*eps)
    return grad

# Approximate Hessian using finite differences
def approximate_hessian(x, eps=1e-5):
    n = len(x)
    hessian = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            x1 = x.copy()
            x2 = x.copy()
            x1[i] += eps
            x1[j] += eps
            x2[i] -= eps
            x2[j] -= eps
            hessian[i,j] = (rosenbrock(x1) - rosenbrock(x2)) / (4*eps**2)
    return hessian

# Line search: Golden Section Search
def golden_section_search(f, x, dx, a, b, tol=1e-5):
    phi = (np.sqrt(5) - 1) / 2
    c = b - phi * (b - a)
    d = a + phi * (b - a)
    while abs(c - d) > tol:
        if f(x + c*dx) < f(x + d*dx):
            b = d
        else:
            a = c
        c = b - phi * (b - a)
        d = a + phi * (b - a)
    return (b + a) / 2

def newton_optimization(initial_point, max_iter=50, line_search=True, approximate=False):
    xk = initial_point.copy()
    n = len(xk)
    for k in range(max_iter):
        if approximate:
            gk = approximate_gradient(xk)
            Hk = approximate_hessian(xk)
        else:
            gk = rosenbrock_gradient(xk)
            Hk = rosenbrock_hessian(xk)

        dxk = np.linalg.solve(Hk, -gk)  # Newton's direction
        
        if line_search:
            alpha = golden_section_search(rosenbrock, xk, dxk, 0, 1)
        else:
            alpha = 1  # Exact Newton's step

        xk = xk + alpha * dxk  # Update current point

    return xk

# Test the optimization
n = 100
initial_point = np.random.rand(n)

start_time = time.time()
x_min_exact = newton_optimization(initial_point, line_search=True, approximate=False)
end_time = time.time()
exact_time = end_time - start_time

start_time = time.time()
x_min_approx = newton_optimization(initial_point, line_search=True, approximate=True)
end_time = time.time()
approx_time = end_time - start_time

print("Solution with exact Newton's step:", x_min_exact)
print("Execution time (exact):", exact_time)

print("Solution with approximate Newton's step:", x_min_approx)
print("Execution time (approximate):", approx_time)


Solution with exact Newton's step: [ 0.59296547  0.45855148  0.02687405  0.44679509  0.71903654  0.53673013
  0.50722827  0.4857465   0.28490419  0.13449919  0.66983319  0.68746915
  0.78546971  0.69645867  0.62537435  0.33661067  0.15099516 -0.15359892
 -0.0342159   0.01870965  0.0165668   0.73699825  1.08483914  1.09930598
  0.90953726  0.59332463  0.06209645 -0.40143229  0.50022293  0.47323492
  0.56574605  0.59743966  0.66307704  0.20055644  0.0508991  -0.02942428
  0.00838037 -0.11728376 -0.44471777  0.39436722  0.38670813  0.50235004
  0.44969146 -0.03280621  0.43542671  0.73344676  0.72178562  0.69541317
  0.5918374   0.40511501  0.22620757 -0.01207019  0.67625627  0.57323953
  0.54795996  0.50150506  0.35162112  0.24416848  0.34099208  0.16437595
  0.60075508  0.83112121  0.83630243  0.74035141  0.66097912  0.46349946
  0.5570748   0.26304678  0.23023656  0.40860016  0.49919843  0.60309163
  0.51863789  0.37089682  0.22844674 -0.06298117  0.04769188 -0.66159085
  0.04740107 -0.